In [1]:
import pandas as pd
import gmaps



from config import gkey

gmaps.configure(api_key=gkey)

In [2]:
country_df = pd.read_csv('country capitals coordinates.csv')
country_df.head()                        

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName,Unnamed: 6
0,Somaliland,Hargeisa,9.55,44.050000,NaN,Africa,NaN
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS,Antarctica,NaN
2,French Southern and Antarctic Lands,Port-aux-Français,-49.35,70.216667,TF,Antarctica,NaN
3,Palestine,Jerusalem,31.76666667,35.233333,PS,Asia,NaN
4,Aland Islands,Mariehamn,60.116667,19.900000,AX,Europe,NaN


In [38]:
capitals_df = capitals.dropna(how='any')

In [5]:
fertility_rates_df = pd.read_csv('csv_files/Age-specific fertility rates, Total fertility and .csv')

fertility_rates_df.head                              

<bound method NDFrame.head of      United Nations, Department of Economic and Social Affairs Unnamed: 1  \
0     Population Division, Fertility and Family Plan...               NaN   
1     World Fertility Data 2012\r\nAGE-SPECIFIC FERT...               NaN   
2                                               Country          ISO code   
3                                                   NaN               NaN   
4                                           Afghanistan                 4   
...                                                 ...               ...   
1072                                           Zimbabwe               716   
1073                                           Zimbabwe               716   
1074                                           Zimbabwe               716   
1075                                           Zimbabwe               716   
1076                                           Zimbabwe               716   

       Unnamed: 2  Unnamed: 3 Unnamed: 4     

In [6]:
total_fertility_df = fertility_rates_df.loc[:, ["United Nations, Department of Economic and Social Affairs", "Unnamed: 5"]]
total_fertility_df.head()


,"United Nations, Department of Economic and Social Affairs",Unnamed: 5
0,"Population Division, Fertility and Family Plan...",NaN
1,World Fertility Data 2012\r\nAGE-SPECIFIC FERT...,NaN
2,Country,Total fertility
3,NaN,NaN
4,Afghanistan,8.21


In [7]:
total_fertility_clean = total_fertility_df.loc[4:1074, "United Nations, Department of Economic and Social Affairs":"Unnamed: 5"]
total_fertility_clean.head(10)


,"United Nations, Department of Economic and Social Affairs",Unnamed: 5
4,Afghanistan,8.21
5,Afghanistan,7.11
6,Afghanistan,7.52
7,Afghanistan,5.11
8,Albania,5.16
9,Albania,3.33
10,Albania,2.86
11,Albania,1.61
12,Albania,1.33
13,Algeria,7.86


In [8]:
total_fertility_renamed = total_fertility_clean.rename(columns={"United Nations, Department of Economic and Social Affairs": "country", "Unnamed: 5": "fertility"})
total_fertility_renamed.head(20) 

,country,fertility
4,Afghanistan,8.21
5,Afghanistan,7.11
6,Afghanistan,7.52
7,Afghanistan,5.11
8,Albania,5.16
9,Albania,3.33
10,Albania,2.86
11,Albania,1.61
12,Albania,1.33
13,Algeria,7.86


In [16]:
clean_total_fertility = total_fertility_renamed.loc[total_fertility_renamed['fertility'] != '..']
clean_total_fertility.head()

,country,fertility
4,Afghanistan,8.21
5,Afghanistan,7.11
6,Afghanistan,7.52
7,Afghanistan,5.11
8,Albania,5.16


In [23]:
clean_total_fertility["fertility"] = clean_total_fertility["fertility"].astype('float')
clean_total_fertility.head()

C:\Users\mzzjo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,country,fertility
4,Afghanistan,8.21
5,Afghanistan,7.11
6,Afghanistan,7.52
7,Afghanistan,5.11
8,Albania,5.16


In [24]:
total_fertility_groups = clean_total_fertility.groupby(["country"])
total_fertility_groups 

In [34]:
unique_countries_fertility = total_fertility_groups["fertility"].mean()
unique_countries_fertility = unique_countries_fertility.reset_index()
unique_countries_fertility

,country,fertility
0,Afghanistan,6.9875
1,Albania,2.8580
2,Algeria,4.5420
3,American Samoa,4.2920
4,Andorra,1.2000
...,...,...
218,Viet Nam,3.4240
219,Wallis and Futuna Islands,3.9500
220,Yemen,6.7980
221,Zambia,6.2440


In [40]:
combined_data_df = unique_countries_fertility.merge(country_df, left_on="country", right_on="CountryName")
combined_data_df.head()

,country,fertility,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName,Unnamed: 6
0,Afghanistan,6.9875,Afghanistan,Kabul,34.51666667,69.183333,AF,Asia,NaN
1,Albania,2.8580,Albania,Tirana,41.31666667,19.816667,AL,Europe,NaN
2,Algeria,4.5420,Algeria,Algiers,36.75,3.050000,DZ,Africa,NaN
3,American Samoa,4.2920,American Samoa,Pago Pago,-14.26666667,-170.700000,AS,Australia,NaN
4,Andorra,1.2000,Andorra,Andorra la Vella,42.5,1.516667,AD,Europe,NaN


In [46]:
fertility = combined_data_df["fertility"]

In [47]:
combined_data_df["CapitalLatitude"] = combined_data_df["CapitalLatitude"].astype('float')
combined_data_df["CapitalLongitude"] = combined_data_df["CapitalLongitude"].astype('float')

In [48]:
capitals = combined_data_df[["CapitalLatitude", "CapitalLongitude"]]

In [63]:
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(capitals, weights=fertility*15,
                                 dissipating=False, 
                                max_intensity=50,
                                 point_radius=2)



fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))